In [1]:
import pandas as pd
import numpy as np

## Reading and Writing Data in Text Format

#### Table: Parsing functions in pandas
| Function | Description |
| :------- | :---------- |
| read_csv | Load delimited data from a file, URL, or file-like object; use comma as default delimiter |
| read_table | Load delimited data from a file, URL, or file-like object; use tab ('\t') as default delimiter |
| read_fwf | Read data in fixed-width column format (i.e., no delimiters) |
| read_clipboard | Version of read_table that reads data from the clipboard; useful for converting tables from web pages |
| read_excel | Read tabular data from an Excel XLS or XLSX file |
| read_hdf | Read HDF5 files written by pandas |
| read_html | Read all tables found in the given HTML document |
| read_json | Read data from a JSON (JavaScript Object Notation) string representation |
| read_msgpack | Read pandas data encoded using the MessagePack binary format |
| read_pickle | Read an arbitrary object stored in Python pickle format |
| read_sas | Read a SAS dataset stored in one of the SAS system’s custom storage formats |
| read_sql | Read the results of a SQL query (using SQLAlchemy) as a pandas DataFrame |
| read_stata | Read a dataset from Stata file format |
| read_feather | Read the Feather binary file format |

In [2]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
pd.read_csv('examples/ex1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
#can use read_table and specified the delimiter
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
#a file will not always have a header row
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
#if thats the case then we can assign ourself or pandas will assign default column names
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('examples/ex2.csv', names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
#if you want to change the 'message' column to be the index of the returned DF then use index_col argument
names=['a','b','c','d','message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In the event that you want to form a hierarchicial index from multiple columns, pass a list of column numbers or names

In [9]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [10]:
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

when a table might not have a fixed delimiter using whitespace or some other pattern to separate fields. In that casem we can pass a regular expression as a delimiter __read_table__. This can be expressed by the __regular expression *\s+*__

In [11]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [12]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result
#read_table infers that the first column should be the DF's index

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [13]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [14]:
#can skip first, third, and fourth rows using skiprows argument
pd.read_csv('examples/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
#missing values are marked by some sentinel value such as NA or NULL
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [16]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [18]:
#na_values option can take rither a list or set of strings to consider missing values
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


#### Table: Some read_csv/read_table function arguments
| Argument | Description |
| :------- | :---------- |
| path | String indicating filesystem location, URL, or file-like object |
| sep or delimiter | Character sequence or regular expression to use to split fields in each row |
| header | Row number to use as column names; defaults to 0 (first row), but should be None if there is no header row |
| index_col | Column numbers or names to use as the row index in the result; can be a single name/number or a list of them for a hierarchical index |
| names | List of column names for result, combine with header=None |
| skiprows | Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip. |
| na_values | Sequence of values to replace with NA. |
| comment | Character(s) to split comments off the end of lines. |
| parse_dates | Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise can specify a list of column numbers or name to parse. If element of list is tuple or list, will combine multiple columns together and parse to date (e.g., if date/time split across two columns). |
| keep_date_col | If joining columns to parse date, keep the joined columns; False by default. |
| converters | Dict containing column number of name mapping to functions (e.g., {'foo': f} would apply the function f to all values in the 'foo' column). |
| dayfirst | When parsing potentially ambiguous dates, treat as international format (e.g., 7/6/2012 -> June 7, 2012); False by default. |
| date_parser | Function to use to parse dates. |
| nrows | Number of rows to read from beginning of file. |
| iterator | Return a TextParser object for reading file piecemeal. |
| chunksize | For iteration, size of file chunks. |
| skip_footer | Number of lines to ignore at end of file. |
| verbose | Print various parser output information, like the number of missing values placed in non-numeric columns. |
| encoding | Text encoding for Unicode (e.g., 'utf-8' for UTF-8 encoded text). |
| squeeze | If the parsed data only contains one column, return a Series. |
| thousands | Separator for thousands (e.g., ',' or '.'). |

### Reading Text Files in Pieces
- When pricessing very large files or figuring out the right set of arguments to correctly process a large file, you may only want to read in a small pirce of a file or iterate through smaller chunks of the file

In [20]:
#make pandas display settings more compact
pd.options.display.max_rows = 10

In [21]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [22]:
#read only a small number of rows
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [23]:
#to read a file in pieces, specify the chunksize as a number of rows
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

The above __TextParser__ object returned by __read_csv__ allows to iterate over the parts of the file according to the __chunksize__.
__TextParser__ is also equipped with a __get_chunk__ method that enables you to read pirces of an arbitrary size.

In [24]:
#we can iterate ex6.csv, aggregating the value counts in the 'key' column
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]

<ipython-input-24-3aad5603e87c>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format
- DF has a method called __to_csv__, to write data out to a comma-separated file but other delimiters can be used

In [25]:
data = pd.read_csv('examples/ex5.csv')

In [26]:
data.to_csv('examples/out1.csv')

In [27]:
!cat examples/out1.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [28]:
import sys
data.to_csv(sys.stdout, sep='|')  #sys.stdout prints the text result to the console

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [29]:
data.to_csv(sys.stdout, na_rep='NULL') #denote the empty strings by some other sentinel value using na_rep argument

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [30]:
#remove row and column label i.e index and header
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [31]:
#only output subset of the columns
data.to_csv(sys.stdout, index=False, columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [32]:
#Series also has to_csv method
dates = pd.date_range('1/1/2001', periods=7)
dates

DatetimeIndex(['2001-01-01', '2001-01-02', '2001-01-03', '2001-01-04',
               '2001-01-05', '2001-01-06', '2001-01-07'],
              dtype='datetime64[ns]', freq='D')

In [33]:
ts = pd.Series(np.arange(7), index=dates)
ts

2001-01-01    0
2001-01-02    1
2001-01-03    2
2001-01-04    3
2001-01-05    4
2001-01-06    5
2001-01-07    6
Freq: D, dtype: int64

In [34]:
ts.to_csv(sys.stdout)

,0
2001-01-01,0
2001-01-02,1
2001-01-03,2
2001-01-04,3
2001-01-05,4
2001-01-06,5
2001-01-07,6


### Working with Delimited Formats
It is not always possible load the data using pandas parsing functions such as pd.read_csv. In some cases, some manual processing may be necessary
- For any file with a single-character delimiter, you can use Python’s built-in csv module. To use it, pass any open file or file-like object to __csv.reader__

In [43]:
import csv
f = open('examples/ex7.csv')
reader = csv.reader(f)

In [36]:
#iterating through the reader like a file yields tuples of values with any quote characters removed
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [37]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [38]:
header, values = lines[0], lines[1:]

In [39]:
data_dict = {h:v for h,v in zip(header, zip(*values))}  #zip(*values) transposes rows to columns
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Define a new format with a different delimiter, string quoting convention, or line terminator, we define a simple subclass of __csv.Dialect__

In [44]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
reader = csv.reader(f,dialect=my_dialect)
reader

Can also give individual CSV dialect parameters as keywords to __csv.reader__ without having to define a subclass

In [45]:
reader = csv.reader(f, delimiter='|')
reader

### JSON Data

In [46]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
{"name": "Katie", "age": 38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [49]:
import json
result = json.loads(obj)  #convert JSON string to Python form
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [50]:
asjson = json.dumps(result)  #convert Python object back to JSON
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [51]:
#passing the converted JSON objects to the DF constructor and subset of the data fields
siblings = pd.DataFrame(result['siblings'], columns=['name','age'])
siblings

,name,age
0,Scott,30
1,Katie,38


__pandas.read_json__ automatically convert JSON datasets in specific arrangement into a Series or DataFrame<br>
Default options for pd.read_json assume that each object in the JSON array is a row in the table

In [52]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [54]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [55]:
#export data from pandas to JSON
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [56]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping
- pd.read_html needs additional library like lxml, beautifulsoup4, html5lib
- pd.read_html function has a number of options, but by default it searches for and attempts to parse all tabular data contained within \<table> tags
- This function uses lxml or Beautiful Soup under the hood to parse data from HTML

In [62]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)

1

In [60]:
failures=tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [63]:
#computing no of bank failures by year
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

## Binary Data Formats
- One way to store data (also known as *serialization* efficientyl in binary format is using Python's built-in __pickle__ serialization
- __to_pickle__ method writes the data to disk in pickle format

In [68]:
frame = pd.read_csv('examples/ex1.csv')
frame.to_pickle('examples/frame_pickle')

In [69]:
#read "pickled" object using pd.read_pickle
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### Some other storage formats
1. HDF5
2. MessagePack
3. bcolz: A compressable column-oriented binary format based on the Blosc compression library.
4. Feather: A cross-language column-oriented file format. Feather uses the Apache Arrow columnar memory format.

### Using HDF5 Format
This file format is intended for storing large quantities of scientific array data. The "HDF" stands for __hierarchical data format__. Each HDF5 file can store multiple datasets and supporting metadata. HDF5 can be a good choice for working with very large datasets that dont't fit into memory, as you ca efficiently read and write small sections of much larger arrays.<br><br>
While it’s possible to directly access HDF5 files using either the PyTables or h5py
libraries, pandas provides a high-level interface that simplifies storing Series and
DataFrame object. The __HDFStore__ class works like a dict and handles the low-level
details

In [71]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [72]:
store['obj1']

,a
0,0.791519
1,-0.345922
2,2.348352
3,-1.337092
4,2.042795
...,...
95,-0.042041
96,1.726939
97,1.002393
98,-0.341772


HDFStore supports two storage schemas __fixed and table__. The latter is generally slower, but it supports query operations using special syntax

In [73]:
store.put('obj2', frame, format='table')

In [74]:
store.select('obj2', where=['index>=10 and index<=15'])

,a
10,0.009104
11,1.658886
12,-0.272850
13,-1.522255
14,-0.912089
15,0.262235


In [75]:
store.close()

In [76]:
frame.to_hdf('mydata.h5', 'obj3',format='table')

In [77]:
pd.read_hdf('mydata.h5', 'obj3', where=['index<6'])

,a
0,0.791519
1,-0.345922
2,2.348352
3,-1.337092
4,2.042795
5,-1.200688


### Reading Microsoft Excel files

In [94]:
#ExcelFile is a class and read_excel is a function
# excel_file = pd.ExcelFile('examples/ex1.xlsx')

In [90]:
#pd.read_excel('examples/ex1.xlsx', 'Sheet1')

To write pandas data to Excel format, you must first create an __ExcelWriter__, then write data to it using pandas objects’ __to_excel__ method

In [92]:
# writer = pd.ExecelWriter('examples.ex2.xlsx')
# frame.to_
excel(writer,'Sheet1')
# writer.save()
# #can also pass file path to to_excel and avoid the ExcelWriter class
# frame.to_excel(examples/ex2.xlsx)

## Interacting with Web APIs
- Access websites pubic APIs from Python using __requests package__

In [95]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [98]:
data = resp.json()  #return a dictionary containing JSON parsed into native Python objects
data[:1]

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/42063',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/42063/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/42063/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/42063/events',
  'html_url': 'https://github.com/pandas-dev/pandas/pull/42063',
  'id': 923284191,
  'node_id': 'MDExOlB1bGxSZXF1ZXN0NjcyMDY1NjMz',
  'number': 42063,
  'title': 'BUG: avoid unnecessary casting in CategoricalIndex.reindex',
  'user': {'login': 'jbrockmendel',
   'id': 8078968,
   'node_id': 'MDQ6VXNlcjgwNzg5Njg=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/8078968?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/jbrockmendel',
   'html_url': 'https://github.com/jbrockmendel',
   'followers_url': 'https://api.github.com/users/jbrockmendel/followers'

In [99]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,42063,BUG: avoid unnecessary casting in CategoricalI...,[],open
1,42062,Backport PR #41875 on branch 1.3.x (BUG: unsta...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,42061,Allow passing custom sentinel into MultiIndex....,[],open
3,42057,"PERF: contiguity, less gil in join algos","[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
4,42056,BUG: pd.Series.argsort misaligned index,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
...,...,...,...,...
25,41998,BUG?: 1.3 behavior change with groupby and asi...,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
26,41997,1.3: (intended?) Behavior change with empty apply,"[{'id': 697792067, 'node_id': 'MDU6TGFiZWw2OTc...",open
27,41996,ENH: Gather `dtype` and `convertors` processin...,"[{'id': 87485152, 'node_id': 'MDU6TGFiZWw4NzQ4...",open
28,41995,[WIP] ENH: `Styler.apply(map)_header` made com...,[],open


## Interacting with Databases

In [100]:
import sqlite3

In [110]:
query = """
    create table test(
    a varchar(100), b varchar(20), c real, d integer)
"""
#create a db connection
con = sqlite3.connect('mydata.sqlite')

In [111]:
con.execute(query)

In [112]:
con.commit()

In [113]:
#insert data
data = [('Atlanta', 'Georgia', 1.25, 6),
('Tallahassee', 'Florida', 2.6, 3),
('Sacramento', 'California', 1.7, 5)]
stmt = "insert into test values(?,?,?,?)"
con.executemany(stmt, data)

In [114]:
con.commit()

In [115]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

Can pass the list of tuples to the DataFrame constructor, but you also need the column names, contained in the cursor’s __description__ attribute

In [116]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [117]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


SQLAlchemy project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases.<br><br>
__read_sql__ function enables to read data easily from a general SQLAlchemy connection

In [121]:
import sqlalchemy as sqla

In [122]:
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
